In [509]:
import pandas as pd
import re

from collections import defaultdict, Counter
from math import log2, sqrt

In [60]:
met = 'meta_Video_Games.json.gz'
rev = 'Video_Games.json.gz'

In [61]:
reviews = pd.read_json(rev, lines=True)

In [62]:
meta = pd.read_json(met, lines=True)

In [63]:
meta.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Video Games, PC, Games]",,[],,Reversi Sensory Challenger,[],,Fidelity Electronics,[],"[>#2,623,937 in Toys &amp; Games (See Top 100 ...",[],Toys &amp; Games,,,,0042000742,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Video Games, Xbox 360, Games, </span></span><...",,[Brand new sealed!],,Medal of Honor: Warfighter - Includes Battlefi...,[B00PADROYW],,by\n \n EA Games,[],"[>#67,231 in Video Games (See Top 100 in Video...","[B0050SY5BM, B072NQJCW5, B000TI836G, B002SRSQ7...",Video Games,,,"\n\t\t\t\t\t\t\t\t\t\t\t\t<span class=""vertica...",0078764343,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Video Games, Retro Gaming & Microconsoles, Su...",,[],,street fighter 2 II turbo super nintendo snes ...,[],,Nintendo,[],"[>#134,433 in Video Games (See Top 100 in Vide...",[],Video Games,,,$0.72,0276425316,[],[],NaN
3,"[Video Games, Xbox 360, Accessories, Controlle...",,[MAS's Pro Xbox 360 Stick (Perfect 360 Stick) ...,,Xbox 360 MAS STICK,[],,by\n \n MAS SYSTEMS,[Original PCB used from Xbox 360 Control Pad (...,"[>#105,263 in Video Games (See Top 100 in Vide...",[],Video Games,,,,0324411812,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,"[Video Games, PC, Games, </span></span></span>...",,"[Phonics Alive! 3, The Speller teaches student...",,Phonics Alive! 3: The Speller,[],,by\n \n Advanced Software Pty. Ltd.,"[Grades 2-12, Spelling Program, Teaches Spelli...","[>#92,397 in Video Games (See Top 100 in Video...",[B000BCZ7U0],Video Games,,,,0439335310,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


In [93]:
reviews

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,1,True,"06 9, 2014",A21ROB4YDOZA5P,0439381673,Mary M. Clark,I used to play this game years ago and loved i...,Did not like this,1402272000,NaN,NaN,NaN
1,3,True,"05 10, 2014",A3TNZ2Q5E7HTHD,0439381673,Sarabatya,The game itself worked great but the story lin...,Almost Perfect,1399680000,NaN,NaN,NaN
2,4,True,"02 7, 2014",A1OKRM3QFEATQO,0439381673,Amazon Customer,I had to learn the hard way after ordering thi...,DOES NOT WORK WITH MAC OS unless it is 10.3 or...,1391731200,15,NaN,NaN
3,1,True,"02 7, 2014",A2XO1JFCNEYV3T,0439381673,ColoradoPartyof5,The product description should state this clea...,does not work on Mac OSX,1391731200,11,NaN,NaN
4,4,True,"01 16, 2014",A19WLPIRHD15TH,0439381673,Karen Robinson,I would recommend this learning game for anyon...,Roughing it,1389830400,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2565344,5,True,"08 1, 2018",ANGB54K3888S4,B01HJEBIAA,josh,"Love it, work good",Works good,1533081600,NaN,NaN,NaN
2565345,5,True,"07 17, 2018",A3TEVKR0ZVQB2T,B01HJEBIAA,Prime Member,I do a lot of copy/paste and other keyboard sh...,Great mouse for work and gaming,1531785600,NaN,NaN,[https://images-na.ssl-images-amazon.com/image...
2565346,5,True,"07 6, 2018",ABE7YPWEHNVJZ,B01HJEBIAA,Amazon Customer,One year in and it's still working great! Hig...,Five Stars,1530835200,NaN,NaN,NaN
2565347,5,True,"06 12, 2018",A3ES9QBK3G192O,B01HJEBIAA,Lina Marmolejos,EXCELENTE,Five Stars,1528761600,NaN,NaN,NaN


In [91]:
asins = set(meta['asin'].tolist())
reviews_cl = reviews.loc[reviews['asin'].isin(asins)]

In [92]:
reviews_cl

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,1,True,"06 9, 2014",A21ROB4YDOZA5P,0439381673,Mary M. Clark,I used to play this game years ago and loved i...,Did not like this,1402272000,NaN,NaN,NaN
1,3,True,"05 10, 2014",A3TNZ2Q5E7HTHD,0439381673,Sarabatya,The game itself worked great but the story lin...,Almost Perfect,1399680000,NaN,NaN,NaN
2,4,True,"02 7, 2014",A1OKRM3QFEATQO,0439381673,Amazon Customer,I had to learn the hard way after ordering thi...,DOES NOT WORK WITH MAC OS unless it is 10.3 or...,1391731200,15,NaN,NaN
3,1,True,"02 7, 2014",A2XO1JFCNEYV3T,0439381673,ColoradoPartyof5,The product description should state this clea...,does not work on Mac OSX,1391731200,11,NaN,NaN
4,4,True,"01 16, 2014",A19WLPIRHD15TH,0439381673,Karen Robinson,I would recommend this learning game for anyon...,Roughing it,1389830400,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2565344,5,True,"08 1, 2018",ANGB54K3888S4,B01HJEBIAA,josh,"Love it, work good",Works good,1533081600,NaN,NaN,NaN
2565345,5,True,"07 17, 2018",A3TEVKR0ZVQB2T,B01HJEBIAA,Prime Member,I do a lot of copy/paste and other keyboard sh...,Great mouse for work and gaming,1531785600,NaN,NaN,[https://images-na.ssl-images-amazon.com/image...
2565346,5,True,"07 6, 2018",ABE7YPWEHNVJZ,B01HJEBIAA,Amazon Customer,One year in and it's still working great! Hig...,Five Stars,1530835200,NaN,NaN,NaN
2565347,5,True,"06 12, 2018",A3ES9QBK3G192O,B01HJEBIAA,Lina Marmolejos,EXCELENTE,Five Stars,1528761600,NaN,NaN,NaN


### Задание 1.


1. Можно попробовать искать в отзывах те слова, которые встречаются в названии товара/бренда (сохраняя порядок). А также самые часто упоминаемые слова в принципе: для этого раздела это "product" "game", "item".
2. Можно искать в отзывах те существительные, которые чаще всего повторяются в этих отзывах (если бы на каждый товар было по многу отзывов, можно было бы смотреть для каждого товара в отдельности).
3. Можно условно считать, что в отзыве слова, написанные с большой буквы -- это названия товара (хотя не все так делают). 

Я выбрала первый подход.

Эти подходы не учитывают случаи, когда используются _this_ или _it_ для отсылки на товар; автоматически сложно определить такую референцию. Также конкретно в этой теме по играм бывает такое, что игра называется, например, _Middle Earth_, а в отзыве пишут о франшизе _Lord of the Rings_. Кроме того, в отзывах часто могут писать аббревиатуры (_LOTR_), которых нет в названии игры -- можно было бы попробовать доставать их из названия (_Grand Theft Auto: Vice City_ -> _GTA VC_), однако и тут всё неоднозначно: иногда служебные части речи пропускаются в аббревиатурах, иногда не полностью название игры становится аббревиатурой (как узнать, где она заканчивается?), и, наконец, в случаях вроде _Playstation 3_ -> _PS3_, где аббревиатура делается из одного слова.

### Задание 2.

In [261]:
title_dict = {}
asins = meta['asin']
titles = meta['title']
assert len(asins) == len(titles)
for i in range(len(asins)):
    title_dict[asins[i]] = titles[i]

In [418]:
def find_title_occurrences(title, review):
    
    occurences, spans = [], []
    punct = ',.:;"()-+!?'
    try:
        title = title.lower()
        review = review.lower()
        title = re.sub(r'(?<! )(?=[,.:;"\(\)\-+!?])|(?<=[,.:;"\(\)\-+!?])(?! )', r' ', title)
        words = title.split()
        reg = r'game(?<=\b)|product(?<=\b)|item(?<=\b)|\b' # commonly found words in reviews
        for word in words:
            if word in punct:
                word = "\\"+word
            reg += f"({word} )?"

        for i in re.finditer(reg, review):
            if len(i.group().strip()) > 2 and i.group().strip() not in ['the', 'and', 'off', 'for', 'one',
                                                                       'with', 'not', 'of the', 'of a',
                                                                       'on the', 'on a', 'new']:
                span = i.span()
                if i.group()[0] == ' ':
                    span = (span[0]+1, span[1])
                if i.group()[1] == ' ':
                    span = (span[0], span[1]-1)
                occurences += [i.group().strip()]
                spans += [span]
                
    except Exception:
        pass

    return (tuple(occurences), tuple(spans))

In [626]:
sample = reviews_cl.sample(1000)
sample['title'] = [title_dict[i] for i in sample['asin']]

reviews_cl['title'] = [title_dict[i] for i in reviews_cl['asin']]

C:\Users\Katya\AppData\Local\Temp/ipykernel_2352/592046693.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_cl['title'] = [title_dict[i] for i in reviews_cl['asin']]


In [556]:
sample['NerOccurrences'] = sample.apply(lambda x: find_title_occurrences(x['title'], x['reviewText']), axis=1)
sample

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image,title,NerOccurrences
1301008,5,True,"01 18, 2014",A3JT3H8JAJ7I6Q,B00D4WTO1M,Veronica V.,I purchased this for my nephews as a Christmas...,Love it!,1390003200,NaN,{'Format:': ' Video Game'},NaN,Call of Duty: Ghosts - PlayStation 4,"((), ())"
1334752,5,True,"10 30, 2014",A1GIE8OYZN28LW,B00DD0B1R0,Preston Johnson,Absolutly wonderfull game. Plenty of characte...,lots of fun.,1414627200,NaN,{'Format:': ' Video Game'},NaN,Super Smash Bros. - Nintendo 3DS,"((game,), ((21, 25),))"
2356733,5,True,"06 14, 2016",A2NAC5DCGY7HV1,B003JD5TYM,M. Dukes,"I was worried about some of the lyrics, but I ...",but I soon found out all bad words are omitted...,1465862400,NaN,{'Format:': ' Video Game'},NaN,Def Jam Rapstar (Bundle) - Nintendo Wii,"((game,), ((130, 134),))"
198020,5,True,"05 8, 2016",A10UMIIKKL8WTB,B00027CXEM,Rogerp407,"got here timely, works great. happy with purc...",Happy,1462665600,NaN,NaN,NaN,Star Wars Knights of the Old Republic II: The ...,"((), ())"
733792,5,False,"09 27, 2009",A3UTDKH4GP5NUM,B002BU0DPE,Daniel E. Walsh,I was more than pleasantly surprised with the ...,Great head-set!,1254009600,3,NaN,NaN,PS3 Ear Force PX21 Gaming Headset,"((headset, game), ((65, 73), (262, 266)))"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018341,1,True,"02 26, 2014",A2JOZD6RIE1VV6,B005Q1YA5O,Davontae,It did not work I want my money back this crap...,It did not work,1393372800,NaN,"{'Edition:': ' Standard', 'Platform:': ' PC Do...",NaN,The Sims 3 Pets [Download],"((), ())"
2387889,4,True,"08 6, 2013",AIEFOIVARLQAU,B005HRZ3N0,Ian,Enjoyed this quality FPS. Length was good with...,Fun but too stealthy at times,1375747200,NaN,NaN,NaN,Deus Ex: Human Revolution - Augmented Edition ...,"((), ())"
342773,4,True,"12 2, 2012",A1S7NYZF69KYAP,B000J14PD8,Peanut Pedro,It's a very good game. The first time I loade...,A very good game,1354406400,NaN,NaN,NaN,Flight Simulator - PC,"((game, flight), ((17, 21), (184, 191)))"
1674003,1,True,"03 13, 2015",ADHUY2C04XWD5,B00MOJXG1K,Michaela Rosenfelt,Who makes a xbox controller with no headset co...,One Star,1426204800,NaN,{'Format:': ' Video Game'},NaN,Rock Candy Wired Controller for Xbox One - Blu...,"((xbox, controller), ((12, 17), (17, 28)))"


In [628]:
reviews_cl['NerOccurences'] = reviews_cl.apply(lambda x: find_title_occurrences(x['title'], x['reviewText']), axis=1)
reviews_cl.head(15)

C:\Users\Katya\AppData\Local\Temp/ipykernel_2352/323294826.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_cl['NerOccurences'] = reviews_cl.apply(lambda x: find_title_occurrences(x['title'], x['reviewText']), axis=1)


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image,title,NerOccurences
0,1,True,"06 9, 2014",A21ROB4YDOZA5P,0439381673,Mary M. Clark,I used to play this game years ago and loved i...,Did not like this,1402272000,NaN,NaN,NaN,BRAND NEW Learning Company Oregon Trail 5th Ed...,"((game,), ((20, 24),))"
1,3,True,"05 10, 2014",A3TNZ2Q5E7HTHD,0439381673,Sarabatya,The game itself worked great but the story lin...,Almost Perfect,1399680000,NaN,NaN,NaN,BRAND NEW Learning Company Oregon Trail 5th Ed...,"((game,), ((4, 8),))"
2,4,True,"02 7, 2014",A1OKRM3QFEATQO,0439381673,Amazon Customer,I had to learn the hard way after ordering thi...,DOES NOT WORK WITH MAC OS unless it is 10.3 or...,1391731200,15,NaN,NaN,BRAND NEW Learning Company Oregon Trail 5th Ed...,"((learning company, game), ((170, 187), (401, ..."
3,1,True,"02 7, 2014",A2XO1JFCNEYV3T,0439381673,ColoradoPartyof5,The product description should state this clea...,does not work on Mac OSX,1391731200,11,NaN,NaN,BRAND NEW Learning Company Oregon Trail 5th Ed...,"((product, product, game), ((4, 11), (76, 83),..."
4,4,True,"01 16, 2014",A19WLPIRHD15TH,0439381673,Karen Robinson,I would recommend this learning game for anyon...,Roughing it,1389830400,NaN,NaN,NaN,BRAND NEW Learning Company Oregon Trail 5th Ed...,"((learning, game, learning, game), ((23, 32), ..."
5,5,True,"01 7, 2014",A1TLA7XXSZMTS7,0439381673,lori smith,Choose your career which sets your money for t...,Love this game! Fun and educational.,1389052800,2,NaN,NaN,BRAND NEW Learning Company Oregon Trail 5th Ed...,"((), ())"
6,3,True,"10 22, 2013",A3I9GK5OO42B0I,0439381673,Edwin Grabill,Would like it more if they had at least update...,Would like it more if they had at least update...,1382400000,2,NaN,NaN,BRAND NEW Learning Company Oregon Trail 5th Ed...,"((new,), ((65, 69),))"
7,5,False,"10 21, 2013",A3TPP95Y9DH3L9,0439381673,MollyMoore,It took a few hours to get this up and running...,Off to a rough start but once it got going the...,1382313600,NaN,NaN,NaN,BRAND NEW Learning Company Oregon Trail 5th Ed...,"((game,), ((295, 299),))"
8,5,True,"10 29, 2012",A19GOZTT15KPG1,0439381673,Dovie,I oredered this for a daughter who is now 33 a...,The Oregon Trail--5th Edition,1351468800,3,NaN,NaN,BRAND NEW Learning Company Oregon Trail 5th Ed...,"((oregon trail, game), ((72, 85), (219, 223)))"
9,5,False,"02 8, 2010",A1441WFJ5KRP7J,0439381673,Zoe,Well I thought since this idem didn't have any...,Great game but why do the rattle snakes love me?,1265587200,NaN,NaN,NaN,BRAND NEW Learning Company Oregon Trail 5th Ed...,"((game, game, game, game), ((91, 95), (171, 17..."


### Задание 3 (n-граммы)

In [631]:
texts = reviews_cl['reviewText'].tolist()
occ = reviews_cl['NerOccurences'].tolist()
assert len(texts) == len(occ)

occ[:10]

[(('game',), ((20, 24),)),
 (('game',), ((4, 8),)),
 (('learning company', 'game'), ((170, 187), (401, 405))),
 (('product', 'product', 'game'), ((4, 11), (76, 83), (113, 117))),
 (('learning', 'game', 'learning', 'game'),
  ((23, 32), (32, 36), (58, 67), (116, 120))),
 ((), ()),
 (('new',), ((65, 69),)),
 (('game',), ((295, 299),)),
 (('oregon trail', 'game'), ((72, 85), (219, 223))),
 (('game', 'game', 'game', 'game'),
  ((91, 95), (171, 175), (594, 598), (952, 956)))]

In [712]:
ngrams = defaultdict(list)

word_counter = Counter()
number_of_bigrams = 0

for i in range(len(texts)):
    text = texts[i]
    if isinstance(text, str):
        text = re.sub(r'\s', ' ', text)
        number_of_bigrams += text.count(' ')
        word_counter.update(re.sub(r',\.:;"', '', text).lower().split())
    spans = occ[i][1]
    occurrences = occ[i][0]
    for _, span in enumerate(spans):
        s, e = span
        ngr_s, ngr_e = 0, len(text)
        for n, j in enumerate(text[s-2:0:-1]):
            if j in (' ', '\n'): 
                ngr_s = s-n-1
                break
        ngrams[occurrences[_].strip()] += [re.sub('[,:."]', '', text[ngr_s:e]).strip('').lower()]
        for n, j in enumerate(text[e+1:]):
            if j in (' ', '\n'):
                ngr_e = n+e+1
                break
        ngrams[occurrences[_].strip()] += [re.sub('[,:."]', '', text[s:ngr_e]).strip('').lower()]
        word_counter.update([occurrences[_].strip()])

In [713]:
total_words = sum(word_counter.values())
total_words, number_of_bigrams

(195456942, 190930840)

In [714]:
for k, v in ngrams.items():
    ngrams[k] = Counter(v)

In [747]:
ngrams['dvd'].most_common(10)

[('dvd player', 343),
 ('a dvd ', 224),
 ('the dvd ', 185),
 ('dvd drive', 65),
 ('dvd players', 36),
 ('this dvd ', 33),
 ('my dvd ', 32),
 ('dvd and', 22),
 ('dvd drives', 22),
 ('new dvd ', 21)]

### Задание 4.

#### PMI

In [715]:
def PMI(count, w1_count, w2_count, total_words):
    if count > 6:
        return log2(count/number_of_bigrams / (w1_count/total_words * w1_count/total_words))
    else:
        return 0

In [716]:
PMIs = {}

for key, ngrams_ in ngrams.items():
    for ngram, count in ngrams_.items():
        
        try:
            PMIs[ngram] = PMI(count, 
                              word_counter[key], 
                              word_counter[ngram.replace(key, '').strip()],
                              total_words)
        except Exception:
            pass
            
PMIs = {k:v for k, v in PMIs.items() if k and v}

In [717]:
[(k,v) for k, v in sorted(PMIs.items(), key=lambda item: item[1])][:10]

[('game (xbox', -14.16091651605012),
 ('game saves!', -14.16091651605012),
 ('husbands game', -14.16091651605012),
 ('game shared', -14.16091651605012),
 ('game layouts', -14.16091651605012),
 ('strategy/rpg game', -14.16091651605012),
 ('gamewhatever', -14.16091651605012),
 ('game swap', -14.16091651605012),
 ('eidos game', -14.16091651605012),
 ('game paved', -14.16091651605012)]

In [718]:
[(k,v) for k, v in sorted(PMIs.items(), key=lambda item: item[1], reverse=True)][:10]

[('007 nightfire', 24.76872133576742),
 ('lunar silver star story', 24.76872133576742),
 ('blood of the sacred', 24.76872133576742),
 ('sacred blood of the', 24.76872133576742),
 ('mario is missing', 24.76872133576742),
 ('game boy printer ', 24.76872133576742),
 ('f-zero maximum velocity ', 24.76872133576742),
 ('vs snk 2 ', 24.76872133576742),
 ('daxter the precursor legacy', 24.76872133576742),
 ('batman dark tomorrow', 24.76872133576742)]

#### t-score

In [719]:
def tscore(bigram_count, w1_count, w2_count, total):
    if bigram_count > 6:
        return (bigram_count/number_of_bigrams-w1_count/total*w2_count/total)/sqrt(bigram_count/number_of_bigrams*(1-(bigram_count/number_of_bigrams))/total)
    else:
        return 0

In [720]:
tscores = {}

for key, ngrams_ in ngrams.items():
    for ngram, count in ngrams_.items():
        
        tscores[ngram] = tscore(count, 
                                word_counter[key], 
                                word_counter[ngram.replace(key, '').strip()],
                               total_words)
        
        
tscores = {k:v for k, v in tscores.items() if len(k.split()) > 1 and v}

In [721]:
[(k,v) for k, v in sorted(tscores.items(), key=lambda item: item[1])][:10]

[('gamea game', -44896.09797081758),
 ('gamethe game', -39355.18564299205),
 ('that the', -27781.424132150965),
 ('and this ', -22150.78187879865),
 ('have the', -18106.333958925545),
 ('to you ', -17178.882721924467),
 ('and that ', -16757.15449897574),
 ('but the', -14902.875788088384),
 ('game that ', -13961.735995087767),
 ('of this ', -11677.21354267972)]

In [722]:
[(k,v) for k, v in sorted(tscores.items(), key=lambda item: item[1], reverse=True)][:10]

[('the game', 673.2001435001475),
 ('game is', 430.5439648289593),
 ('this mouse ', 171.62924549893324),
 ('game play', 166.60814352476498),
 ('game has', 166.31592265898547),
 ('the mouse ', 141.2297292589814),
 ('game itself', 125.2830676867813),
 ('game was', 118.7690701488378),
 ('this keyboard ', 116.87538184269863),
 ('this headset ', 115.52629305664041)]

#### chi-square

In [723]:
def chi2(bigram_count, w1_count, w2_count, total):
    
    if bigram_count > 4:

        A = total * (bigram_count * (total-w1_count-w2_count+bigram_count) - (bigram_count-w1_count) * (bigram_count-w2_count))
        B = w1_count * w2_count * (bigram_count-w1_count + total-w1_count-w2_count+bigram_count) * (bigram_count-w2_count + total-w1_count-w2_count+bigram_count)

        try:
            return A/B
        except Exception:
            return None
    else:
        return 0

In [724]:
chi2s = {}

for key, ngrams_ in ngrams.items():
    for ngram, count in ngrams_.items():
        
        chi2s[ngram] = chi2(count, 
                            (word_counter[key]), 
                            (word_counter[ngram.replace(key, '').strip()]),
                               total_words)
        
chi2s = {k:v for k, v in chi2s.items() if k and v}

In [725]:
[(k,v) for k, v in sorted(chi2s.items(), key=lambda item: item[1])][:10]

[('thegame', -6.550816250867923e-09),
 ('gamethe game', -6.550437673306045e-09),
 ('gamethe', -6.5395599902223055e-09),
 ('that the', -6.197435810298198e-09),
 ('but the', -6.166199049482841e-09),
 ('have the', -6.1474771429158005e-09),
 ('like the', -6.0917937528581634e-09),
 ('the all ', -6.08200282885606e-09),
 ('great the', -6.07738051975682e-09),
 ('get the', -6.074702732815047e-09)]

In [726]:
[(k,v) for k, v in sorted(chi2s.items(), key=lambda item: item[1], reverse=True)][:10]

[('vs street', 0.9999999846513512),
 ('castlevania symphony of the', 0.9999999846513512),
 ("oddworld abe's", 0.9999999846513512),
 ("oddworld munch's oddysee", 0.9999999846513512),
 ('batman rise of sin', 0.9999999846513512),
 ('moon a wonderful life', 0.9999999846513512),
 ('order criminal intent', 0.9999999846513512),
 ('spartan total warrior', 0.9999999846513512),
 ('wars dual strike', 0.9999999846513512),
 ('ocean blue world', 0.9999999846513512)]

Результаты хи-квадрат похожи на результаты PMI; t-score склонаяется к _game_, для которого больше всех данных (это слово использовалось во всех отзывах вне зависимости от продукта). Мне больше всего нравится результат PMI.

### Задание 5.

In [751]:
[(k,v.most_common(5)) for k,v in sorted(ngrams.items(), key=lambda x: len(x[1]), reverse=True)][:10]

[('game',
  [('the game', 876648),
   ('this game', 876289),
   ('game', 752911),
   ('game is', 324038),
   ('a game', 118048)]),
 ('product',
  [('product', 70324),
   ('this product', 60999),
   ('the product', 35678),
   ('great product', 25924),
   ('data-hook=product', 14126)]),
 ('mouse',
  [('the mouse ', 45311),
   ('this mouse ', 36576),
   ('mouse is', 19104),
   ('mouse for', 8260),
   ('mouse and', 6377)]),
 ('xbox',
  [('the xbox ', 20623),
   ('xbox 360', 10232),
   ('xbox one', 9486),
   ('my xbox ', 8540),
   ('xbox live', 8456)]),
 ('controller',
  [('the controller ', 26898),
   ('this controller ', 15597),
   ('controller is', 9517),
   ('controller and', 5703),
   ('controller to', 3229)]),
 ('item',
  [('this item', 19938),
   ('item', 15261),
   ('the item', 11661),
   ('', 5084),
   ('item was', 4503)]),
 ('wii',
  [('the wii ', 28353),
   ('wii remote', 4977),
   ('my wii ', 4818),
   ('wii u', 4773),
   ('a wii ', 4683)]),
 ('keyboard',
  [('this keyboard ', 1

In [758]:
keys = ['mouse', 'xbox', 'controller', 'wii', 'keyboard']
subdict = {k:ngrams[k] for k in keys}

for key, ngrams_ in subdict.items():
    PMIs = {}
    for ngram, count in ngrams_.items():
        
        try:
            PMIs[ngram] = PMI(count, 
                              word_counter[key], 
                              word_counter[ngram.replace(key, '').strip()],
                              total_words)
        except Exception:
            pass
            
    PMIs = {k:v for k, v in PMIs.items() if k and v}
    print(key, 'PMI:')
    for i in [(k,v) for k, v in sorted(PMIs.items(), key=lambda item: item[1], reverse=True)][:10]:
        print(i)

mouse PMI:
('the mouse ', 6.561908954774108)
('this mouse ', 6.252944932395364)
('mouse is', 5.31592236544569)
('mouse for', 4.106261310487693)
('mouse and', 3.7329974100405403)
('a mouse ', 3.3567237679319035)
('mouse to', 3.3053109219072807)
('mouse i', 3.0027278649546356)
('mouse that', 2.9868780608294374)
('mouse has', 2.961613983890547)
xbox PMI:
('the xbox ', 6.247359070361973)
('xbox 360', 5.236193024819999)
('xbox one', 5.126976629017888)
('my xbox ', 4.975412830446214)
('xbox live', 4.9611521373210445)
('an xbox ', 4.069577685203659)
('on xbox ', 3.839462661523706)
('xbox and', 3.7191200024914757)
('xbox controller', 3.2795271309464136)
('for xbox ', 3.162133074431505)
controller PMI:
('the controller ', 6.30461459549634)
('this controller ', 5.518384251326978)
('controller is', 4.80569446648155)
('controller and', 4.066908628094204)
('controller to', 3.2462750363384765)
('my controller ', 3.1309839707156644)
('a controller ', 3.104610341133246)
('controller for', 3.0907429287